#### Notebook for running X-CER experiments on IBM Quantum Platform Devices

##### **Things to Check and Remember Before Running This Notebook**

1. Make sure IBM Quantum token access is enabled.
2. Ensure all imports (`xcer_funcs` especially) are imported
3. Save circuits before, during, and after execution
4. Set max_submissions high

In [7]:
##### Imports #####
# xcer_funcs contains functions for generating the X-CER circuits
# We also want to import the Qiskit IBM Provider
from xcer_funcs import generate_cer_circuits
from datalogger import DataLogger

from qiskit_ibm_runtime import QiskitRuntimeService

# Import numpy and trueq
import numpy as np
import trueq as tq

import datetime
import random

**Picking an IBM Backend**

We've been using:

1. ibm_cusco
2. ibm_sherbrooke
3. ibm_kyiv
3. ibm_torino

In [8]:
service = QiskitRuntimeService(instance="ibm-q-ncsu/nc-state/noiseanderror")

backend = service.backend('ibm_kyiv')

KeyboardInterrupt: 

In [12]:
# Generate a unique identifer for bookeeping purposes
id = str(random.randint(0, 1000000))
print(F"ID for these simulations: {id}\nUse this ID to identify results folder.")


results_directory = F"./devices/{backend.name}/{id}/"

dl = DataLogger(directory=results_directory, id=id)

experiments_info = {"id": id, 
                    "backend": backend.name, 
                    "date": datetime.datetime.now().strftime("%Y-%m-%d")}

ID for these simulations: 13077
Use this ID to identify results folder.


In [ ]:
# Define the parameters for the experiment
# Using sequence lengths of 4, 8, 12 for coherent errors and cnot repetitions 4, 8, 12
rep_len_tuples = [(1, 4), (4, 4), (6, 4), (1, 2), (8, 2), (12, 2), (1, 6), (2, 6), (4, 6)]

# CAN UNCOMMENT THIS LINE TO TEST A SMALLER NUMBER OF CIRCUITS
rep_len_tuples = [(6, 4)]

# Define the hard cycle of interest
# the cycle is a CNOT on qubits 0,1 and an identity gate on ancillary qubit 2
cycle = tq.Cycle({(0, 1): tq.Gate.cx, 2: tq.Gate.id})

# Number of random circuits per 
n_randomizations = 450

# Generate the CER circuits
# pass in the cycle, repetitions, and sequence_lengths
cer_circuits = generate_cer_circuits(cycle, rep_len_tuples, n_circuits=n_randomizations)

# We can change the batch size here
batch_size = 225
num_batches = len(cer_circuits) // batch_size
print(f"Number of batches: {num_batches}")
print(F"Total number of CER circuits: {len(cer_circuits)}")

# Draw our hard cycle of interest
print("Hard cycle of interest:")
cycle_circ = tq.Circuit(cycles=[cycle])
cycle_circ.draw()

# Gather circuit info, save circuits to disk
experiments_info['batch_size'] = batch_size
experiments_info['num_batches'] = num_batches
experiments_info['num_circuits'] = len(cer_circuits)
experiments_info['rep_len_tuples'] = rep_len_tuples
experiments_info["n_randomizations"] = n_randomizations

# Save circuits before execution
cer_circuits.save(f"./devices/{backend.name}/{id}/unexecuted_cer_circuits.tq")

Number of batches: 6
Total number of CER circuits: 1350
Hard cycle of interest:


### We will not worry about qubit relabeling at this moment

In [9]:
# # Guadalupe physical qubits: 6-7 10 12 13
# # Cairo physical qubits: 17-18 21 23 24
# # sherbrooke physical qubits: 100 110 118 119 120
# relabels = {"ibmq_guadalupe": {0: 6, 1: 7, 2: 10, 3: 12, 4: 13},
#             "ibm_cairo": {0: 17, 1: 18, 2: 21, 3: 23, 4: 24},
#             "ibm_sherbrooke": {0: 100, 1: 110, 2: 118, 3: 119, 4: 120}}


# from trueq.compilation import Relabel
# relabel = Relabel(relabels[backend.name])
# cer_circuits = tq.compilation.Compiler([relabel]).compile(cer_circuits)

# rcal_circuits = tq.make_rcal(cer_circuits.labels)

# # save physical qubits mapping
# experiments_info['physical_qubits'] = [(key, value) for key, value in relabels[backend.name].items()]



### Create RCAL circuits to append for readout calibration

In [ ]:
rcal_circuits = tq.make_rcal(cer_circuits.labels)

In [10]:
# Save experiments info
dl.write_data(experiments_info)

In [ ]:
batches = cer_circuits.batch(batch_size, extra_circuits=rcal_circuits, sequencer=tq.sequencer.RIFFLER)

In [ ]:
ex = tq.interface.Executor(cer_circuits, backend, filename=F"./devices/{backend.name}/{id}/executing_circuits.tq", n_shots=10000, max_submissions=100)

In [ ]:
ex.circuits.save(F"./devices/{backend.name}/{id}/executed_circuits.tq")